In [ ]:
import pandas as pd
from sklearn import tree
import shap
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import neighbors
from tqdm import tqdm
import optuna
from sklearn import datasets
import sklearn
import warnings
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
sklearn.metrics.SCORERS.keys()

# Задание 1

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
df.sample(2)

In [ ]:
df.info()

In [ ]:
df_new = df[['Pclass','Fare','Age','Sex','Survived']].dropna()
# df_new = pd.get_dummies(df_new)

In [ ]:
X = df_new.drop('Survived',axis=1)
y = df_new[['Survived']]

In [ ]:
X.Sex = preprocessing.LabelEncoder().fit_transform(X.Sex)

In [ ]:
X.sample(2)

In [ ]:
model = tree.DecisionTreeClassifier(random_state=100)
model.fit(X,y)

In [ ]:
shap_test = shap.TreeExplainer(model).shap_values(X)
shap.summary_plot(shap_test, X,
                      max_display=25, auto_size_plot=True)

In [ ]:
tree.plot_tree(model)
print()

# Задание 2

In [ ]:
df = pd.read_csv('wine.csv',header=None)

In [ ]:
df.head(2)

In [ ]:
y = df[[0]]
X = df.drop(0,axis=1)

In [ ]:
df.info()

In [ ]:
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=100).split(X)

In [ ]:
%%time
for train, test in tqdm(kfold):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    model = neighbors.KNeighborsClassifier()
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))

In [ ]:
model_selection.cross_val_score(neighbors.KNeighborsClassifier(),X,y)

In [ ]:
# def obj(self, trial):
#         model = cb.CatBoostClassifier(
#         iterations=trial.suggest_int("iterations", 1, 1000),
#         learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
#         depth=trial.suggest_int("depth", 4, 10),
#         l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
#         bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
#         random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
#         bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
#         od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
#         od_wait=trial.suggest_int("od_wait", 10, 50),
#         verbose=False
#     )
#         model.fit(self.X_train, self.y_train)
#         self.catboost_model = model
#         y_pred = model.predict(self.X_test)
#         return accuracy_score(self.y_test, y_pred)
    
#     def test(self):
#         study = optuna.create_study(study_name="catboost", direction="maximize")
#         study.optimize(self.obj, n_trials=5, n_jobs=-1, callbacks=[self.callback])

In [ ]:
def obj(trial):
    model = neighbors.KNeighborsClassifier(n_neighbors=trial.suggest_int("n_neighbors",1,100))
    return min(model_selection.cross_val_score(model,X,y,scoring='accuracy',n_jobs=-1))
study = optuna.create_study(direction='maximize')
study.optimize(obj,n_trials=100,n_jobs=-1)

In [ ]:
print(study.best_params)

In [ ]:
print(study.best_value)

In [ ]:
X = preprocessing.scale(X)

In [ ]:
def obj(trial):
    model = neighbors.KNeighborsClassifier(n_neighbors=trial.suggest_int("n_neighbors",1,100))
    return min(model_selection.cross_val_score(model,X,y,scoring='accuracy',n_jobs=-1))
study = optuna.create_study(direction='maximize')
study.optimize(obj,n_trials=100,n_jobs=-1)

In [ ]:
print(study.best_params)

In [ ]:
print(study.best_value)

# Задание 3

In [ ]:
X, y = datasets.load_boston(return_X_y=True)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X = preprocessing.scale(X)

In [ ]:
def obj(trial):
    model = neighbors.KNeighborsRegressor(n_neighbors=6, weights='distance', p=trial.suggest_int('p',1,20),n_jobs=-1)
    return max(map(lambda x: -1*x, model_selection.cross_val_score(model,X,y,scoring='neg_mean_squared_error',n_jobs=-1,cv=10)))
study = optuna.create_study(direction='minimize')
study.optimize(obj,n_trials=100,n_jobs=-1)

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
from s